In [17]:
import sys
sys.path.append("../")
from backpain_helper import BackpainHelper
from sklearn import datasets, neighbors, metrics,grid_search, model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.io import sql
import sqlite3
%matplotlib inline

bh = BackpainHelper()
df = bh.get_spine_data()

columns = ['pelvic_incidence', 'pelvic_tilt','lumbar_lordosis_angle','sacral_slope','pelvic_radius','degree_spondylolisthesis','pelvic_slope','direct_tilt','thoracic_slope','cervical_tilt','sacrum_angle','scoliosis_slope']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df[columns], df.classification, test_size=0.33, random_state=42)
knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
print accuracy_score(y_test, pred)

x = [i for i in range(1, 50)]
kf = cross_validation.KFold(len(df), n_folds = 5, shuffle=True)
gs = grid_search.GridSearchCV(
    estimator=neighbors.KNeighborsClassifier(),
    param_grid={'n_neighbors': x, 'weights':['uniform', 'distance']},
    cv=kf
)
gs.fit(df[columns], df.classification)
print gs.grid_scores_
print gs.best_score_

0.805825242718
[mean: 0.78710, std: 0.06238, params: {'n_neighbors': 1, 'weights': 'uniform'}, mean: 0.78710, std: 0.06238, params: {'n_neighbors': 1, 'weights': 'distance'}, mean: 0.76452, std: 0.06658, params: {'n_neighbors': 2, 'weights': 'uniform'}, mean: 0.78710, std: 0.06238, params: {'n_neighbors': 2, 'weights': 'distance'}, mean: 0.82581, std: 0.04003, params: {'n_neighbors': 3, 'weights': 'uniform'}, mean: 0.82581, std: 0.04003, params: {'n_neighbors': 3, 'weights': 'distance'}, mean: 0.82258, std: 0.05680, params: {'n_neighbors': 4, 'weights': 'uniform'}, mean: 0.82581, std: 0.05983, params: {'n_neighbors': 4, 'weights': 'distance'}, mean: 0.83548, std: 0.04493, params: {'n_neighbors': 5, 'weights': 'uniform'}, mean: 0.83226, std: 0.04280, params: {'n_neighbors': 5, 'weights': 'distance'}, mean: 0.82581, std: 0.05141, params: {'n_neighbors': 6, 'weights': 'uniform'}, mean: 0.84194, std: 0.04131, params: {'n_neighbors': 6, 'weights': 'distance'}, mean: 0.81613, std: 0.04741, p

In [26]:
from sklearn.tree import export_graphviz
from os import system 
def build_tree_image(model):
    dotfile = open("tree.dot", 'w')
    export_graphviz(model,
                              out_file = dotfile,
                              feature_names = columns)
    dotfile.close()
    system("dot -Tpng tree.dot -o tree.png")
    

model = DecisionTreeClassifier(
                max_depth = 16,
                min_samples_leaf = 5)

model.fit(df[columns], df.classification)
build_tree_image(model)
features = df[columns]
feature_importances = model.feature_importances_
print feature_importances
features_df = pd.DataFrame({'Features': columns, 'Importance Score': feature_importances})
features_df.sort_values('Importance Score', inplace=True, ascending=False)
features_df.head(12)


[ 0.          0.04803343  0.02249932  0.13045608  0.1201284   0.52993917
  0.          0.01020907  0.          0.00365886  0.13507567  0.        ]


,Features,Importance Score
5,degree_spondylolisthesis,0.529939
10,sacrum_angle,0.135076
3,sacral_slope,0.130456
4,pelvic_radius,0.120128
1,pelvic_tilt,0.048033
2,lumbar_lordosis_angle,0.022499
7,direct_tilt,0.010209
9,cervical_tilt,0.003659
0,pelvic_incidence,0.000000
6,pelvic_slope,0.000000


In [27]:
model = RandomForestClassifier(n_estimators = 20)
    
model.fit(df[columns], df.classification)

kf = cross_validation.KFold(len(df), n_folds = 5, shuffle=True)
gs = grid_search.GridSearchCV(
    estimator=neighbors.KNeighborsClassifier(),
    param_grid={'n_neighbors': x, 'weights':['uniform', 'distance']},
    cv=kf
)
gs.fit(df[columns], df.classification)
print gs.grid_scores_
print gs.best_score_

[mean: 0.80645, std: 0.04446, params: {'n_neighbors': 1, 'weights': 'uniform'}, mean: 0.80645, std: 0.04446, params: {'n_neighbors': 1, 'weights': 'distance'}, mean: 0.78710, std: 0.06238, params: {'n_neighbors': 2, 'weights': 'uniform'}, mean: 0.80645, std: 0.04446, params: {'n_neighbors': 2, 'weights': 'distance'}, mean: 0.80968, std: 0.04376, params: {'n_neighbors': 3, 'weights': 'uniform'}, mean: 0.80968, std: 0.04376, params: {'n_neighbors': 3, 'weights': 'distance'}, mean: 0.80323, std: 0.04607, params: {'n_neighbors': 4, 'weights': 'uniform'}, mean: 0.80968, std: 0.03444, params: {'n_neighbors': 4, 'weights': 'distance'}, mean: 0.80323, std: 0.04719, params: {'n_neighbors': 5, 'weights': 'uniform'}, mean: 0.80000, std: 0.04849, params: {'n_neighbors': 5, 'weights': 'distance'}, mean: 0.80645, std: 0.03951, params: {'n_neighbors': 6, 'weights': 'uniform'}, mean: 0.80000, std: 0.03762, params: {'n_neighbors': 6, 'weights': 'distance'}, mean: 0.83871, std: 0.05301, params: {'n_neig

In [31]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(df[columns], df.classification)

kf = cross_validation.KFold(len(df), n_folds = 5, shuffle=True)
gs = grid_search.GridSearchCV(
    estimator=LogisticRegression(),
    param_grid={},
    cv=kf
)
gs.fit(df[columns], df.classification)
print gs.grid_scores_
print gs.best_score_

[mean: 0.82258, std: 0.03678, params: {}]
0.822580645161


In [37]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

scaler = StandardScaler()
logistic_model = LogisticRegression()

modeling_pipe = Pipeline([('scale',scaler),('model',logistic_model)])
modeling_pipe.set_params(model__C = 1).fit(X_train,y_train)
modeling_pipe.predict(X_train)
modeling_pipe.predict(X_test)
modeling_pipe.predict_proba(X_test)

array([[  7.40108067e-01,   2.59891933e-01],
       [  1.58300321e-01,   8.41699679e-01],
       [  5.96067933e-01,   4.03932067e-01],
       [  3.25906749e-02,   9.67409325e-01],
       [  1.82150474e-01,   8.17849526e-01],
       [  2.15064255e-01,   7.84935745e-01],
       [  1.65644815e-02,   9.83435518e-01],
       [  1.74533656e-01,   8.25466344e-01],
       [  2.80510896e-01,   7.19489104e-01],
       [  2.14365465e-01,   7.85634535e-01],
       [  4.68030891e-06,   9.99995320e-01],
       [  8.57843992e-04,   9.99142156e-01],
       [  7.21007857e-01,   2.78992143e-01],
       [  7.68049838e-01,   2.31950162e-01],
       [  6.61654579e-01,   3.38345421e-01],
       [  5.42765645e-01,   4.57234355e-01],
       [  6.46046007e-01,   3.53953993e-01],
       [  8.37914301e-03,   9.91620857e-01],
       [  2.92386000e-01,   7.07614000e-01],
       [  4.62711911e-03,   9.95372881e-01],
       [  3.44047483e-02,   9.65595252e-01],
       [  3.33570292e-04,   9.99666430e-01],
       [  

In [87]:
df = bh.get_spine_data()
X_data = df[columns]
Y_data = df.classification

NUM_TRIALS = 50
# Arrays to store scores
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

# Loop for each trial
for i in range(NUM_TRIALS):
    inner_cv = cross_validation.KFold(len(df), n_folds = 4, shuffle=True, random_state=i)
    outer_cv = cross_validation.KFold(len(df), n_folds = 4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = grid_search.GridSearchCV(estimator=neighbors.KNeighborsClassifier(), 
                       param_grid={'n_neighbors':[5],'weights':['uniform']}, 
                       cv=inner_cv)
    clf.fit(X_data, Y_data)
    non_nested_scores[i] = clf.best_score_
    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=X_data, y=Y_data, cv=outer_cv)
#     nested_scores[i] = nested_score.mean()

score_difference = non_nested_scores - nested_scores
print score_difference.mean()

IndexError: positional indexers are out-of-bounds

In [88]:
df = bh.get_spine_data()
X_data = df[columns]
Y_data = df.classification

NUM_TRIALS = 50
# Arrays to store scores
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

# Loop for each trial
for i in range(NUM_TRIALS):
    inner_cv = cross_validation.KFold(len(df), n_folds = 4, shuffle=True, random_state=i)
    outer_cv = cross_validation.KFold(len(df), n_folds = 4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = grid_search.GridSearchCV(estimator=LogisticRegression(), 
                       param_grid={}, 
                       cv=inner_cv)
    clf.fit(X_data, Y_data)
    non_nested_scores[i] = clf.best_score_
    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=X_data, y=Y_data, cv=outer_cv)
#     nested_scores[i] = nested_score.mean()

score_difference = non_nested_scores - nested_scores
print score_difference.mean()

IndexError: positional indexers are out-of-bounds